In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")


orginal_data_size = len(ratings)
ratings.head()


,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:

# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')


orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


### 일단 ratings 값은 3 이상이면 평균이라 판단하고 추천할 때 3도 추천하게 하자

맘 같아서는 2점 이하 추천 모델을 따로 만들어서
<br/>
3점 이상의 영화를 추천하는 모델이 어떤 유저에게 영화를 추천하기 전에
<br/>
만약 2점 영화를 추천하는 모델과 결과가 같으면 추천하지 않게 하는 용도로 쓰고 싶어서다

<br/>

근데 추천하는 알고리즘이 같다면, 3점 이상 영화를 추천하는 모델과 2점 이하 영화를 추천하는 모델과 겹치는 일은 없을 거 같은데?

### 사람은 손해에 민감하다
<br/>
즉 부정적인 평가를 남긴 사람들은 자신이 어째서 부정적인 평가를 남기게 되었는지에 대해 기록한 데이터가 많이 존재할 거다

즉 알고리즘을 다르게 할 여지가 많다

데이터가 많지 않다 하더라도, 긍정과 부정을 나눠 서로 다른 알고리즘을 가지게 해서 좀 더 정확하게 추천하게 하는 게 결코 의미없는 행동은 아니라고 생각한다
<br/>

*약간 앙상블 느낌이 난다*


In [4]:
print(max(ratings['movie_id']))
print(ratings.loc[ratings["movie_id"]==3952])
print(ratings.loc[ratings["movie_id"]==3953])

3952
        user_id  movie_id  ratings   timestamp
2898         23      3952        4   978461000
5086         36      3952        5   978062904
7310         52      3952        4   977947102
10077        72      3952        5   977868330
13099       102      3952        3  1039274093
...         ...       ...      ...         ...
941592     5682      3952        3  1029457829
964309     5812      3952        4   992072099
967122     5831      3952        3   986223125
968244     5837      3952        4  1011902656
992932     5998      3952        4  1001781044

[353 rows x 4 columns]
Empty DataFrame
Columns: [user_id, movie_id, ratings, timestamp]
Index: []


In [5]:
# 해당 코드를 실행하면 많은 시간이 필요하므로 주의하자

r_movie = []

for i in ratings['movie_id']:
    movies_idx = 0
    for j in movies['movie_id']:
        if i == j:
            r_movie.append(movies['title'][movies_idx])
        else:
            movies_idx+=1
            continue

print(r_movie[:10])

["One Flew Over the Cuckoo's Nest (1975)", 'James and the Giant Peach (1996)', 'My Fair Lady (1964)', 'Erin Brockovich (2000)', "Bug's Life, A (1998)", 'Princess Bride, The (1987)', 'Ben-Hur (1959)', 'Christmas Story, A (1983)', 'Snow White and the Seven Dwarfs (1937)', 'Wizard of Oz, The (1939)']


In [6]:
print(len(r_movie))
print(len(ratings['movie_id']))

836478
836478


In [7]:
# 해당 코드를 실행하면 많은 시간이 필요하므로 주의하자

r_genre = []

for i in ratings['movie_id']:
    genre_idx = 0
    for j in movies['movie_id']:
        if i == j:
            r_genre.append(movies['genre'][genre_idx])
        else:
            genre_idx+=1
            continue

print(r_genre[:10])

['Drama', "Animation|Children's|Musical", 'Musical|Romance', 'Drama', "Animation|Children's|Comedy", 'Action|Adventure|Comedy|Romance', 'Action|Adventure|Drama', 'Comedy|Drama', "Animation|Children's|Musical", "Adventure|Children's|Drama|Musical"]


In [8]:
print(len(movies['movie_id']))
print(len(ratings['user_id']))

print(movies.loc[movies["movie_id"]==1193])
print(movies.loc[movies["movie_id"]==661])
print(movies.loc[movies["movie_id"]==914])
print(movies.loc[movies["movie_id"]==3408])
print(movies.loc[movies["movie_id"]==2355])

print(len(r_movie))
print(len(r_genre))

3883
836478
      movie_id                                   title  genre
1176      1193  One Flew Over the Cuckoo's Nest (1975)  Drama
     movie_id                             title                         genre
655       661  James and the Giant Peach (1996)  Animation|Children's|Musical
     movie_id                title            genre
902       914  My Fair Lady (1964)  Musical|Romance
      movie_id                   title  genre
3339      3408  Erin Brockovich (2000)  Drama
      movie_id                 title                        genre
2286      2355  Bug's Life, A (1998)  Animation|Children's|Comedy
836478
836478


df.loc[df[a]==b] 사용법을 서술한 글

https://www.delftstack.com/ko/howto/python-pandas/pandas-get-index-of-row/

df[a]의 값 중 b와 같은 값을 가진 행렬을 출력하라는 뜻이다

In [9]:
n_data = pd.DataFrame(ratings['user_id'])
print(len(n_data))

print(len(r_movie))
print(len(r_genre))

print(len(movies))

print(n_data.tail())

#n_data{'movie':r_movie,'genre':r_genre}

836478
836478
836478
3883
         user_id
1000203     6040
1000205     6040
1000206     6040
1000207     6040
1000208     6040


In [10]:
n_data['title']=r_movie
n_data['genre']=r_genre
n_data.head()

,user_id,title,genre
0,1,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,My Fair Lady (1964),Musical|Romance
3,1,Erin Brockovich (2000),Drama
4,1,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [11]:
n_data.user_id

0             1
1             1
2             1
3             1
4             1
           ... 
1000203    6040
1000205    6040
1000206    6040
1000207    6040
1000208    6040
Name: user_id, Length: 836478, dtype: int64

In [12]:
print(len(n_data['genre'].unique()))

gen_num = list(n_data['genre'].unique())

301


In [13]:
# 해당 코드를 실행하면 많은 시간이 필요하므로 주의하자

r_genre_n = []

for i in ratings['movie_id']:
    genre_n_idx = 0
    for j in movies['movie_id']:
        if i == j:
            gen_num_c = 0
            for k in gen_num:
                if movies['genre'][genre_n_idx] == k:
                    r_genre_n.append(gen_num_c)
                else:
                    gen_num_c +=1
        else:
            genre_n_idx+=1
            continue

print(r_genre_n[:10])

[0, 1, 2, 0, 3, 4, 5, 6, 1, 7]


csr_matrix((a, (b,c)), shape= (num_b, num_c))

해당 코드에서 a는 정량적으로 해석할 수 있는 데이터다
<br/>
b와 c는 고유벡터 즉 정량적으로 해석할 수 없는 데이터다

때문에 정략적인 데이터를 만들 필요가 있었다

그럼 어떤 데이터가 정략적 해석을 할 수 있는 데이터인가?
<br/>
총 영화 조회수

In [14]:
views = []

for i in n_data['title'].unique():
    views.append(len(n_data.loc[n_data['title']==i]))

In [15]:
print(len(n_data))
print(n_data['title'][:10])
print(n_data['title'].nunique())
print(len(views))
print(views[:10])

836478
0    One Flew Over the Cuckoo's Nest (1975)
1          James and the Giant Peach (1996)
2                       My Fair Lady (1964)
3                    Erin Brockovich (2000)
4                      Bug's Life, A (1998)
5                Princess Bride, The (1987)
6                            Ben-Hur (1959)
7                 Christmas Story, A (1983)
8    Snow White and the Seven Dwarfs (1937)
9                  Wizard of Oz, The (1939)
Name: title, dtype: object
3628
3628
[1680, 443, 611, 1223, 1599, 2252, 666, 1290, 707, 1650]


총 영화는 3628개이며,
<br/>
n_data['title'] 순서로 조회수를 기록했다

<br/>

그럼 이제 영화에 맞게 조회수를 기입하자

In [16]:
aaaa = n_data.loc[n_data['title']==list(set(n_data['title']))[0]]

aaaa['views'] = 4

print(aaaa['views'])
print(aaaa)

7889      4
19557     4
30625     4
39617     4
52226     4
         ..
978191    4
979392    4
980833    4
993392    4
995286    4
Name: views, Length: 218, dtype: int64
        user_id                                              title  \
7889         53  City Slickers II: The Legend of Curly's Gold (...   
19557       148  City Slickers II: The Legend of Curly's Gold (...   
30625       204  City Slickers II: The Legend of Curly's Gold (...   
39617       268  City Slickers II: The Legend of Curly's Gold (...   
52226       343  City Slickers II: The Legend of Curly's Gold (...   
...         ...                                                ...   
978191     5894  City Slickers II: The Legend of Curly's Gold (...   
979392     5908  City Slickers II: The Legend of Curly's Gold (...   
980833     5916  City Slickers II: The Legend of Curly's Gold (...   
993392     6000  City Slickers II: The Legend of Curly's Gold (...   
995286     6010  City Slickers II: The Legend of Curly's Go

/tmp/ipykernel_67/672558532.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aaaa['views'] = 4


In [17]:
print(len(set(n_data['title'])))
print(list(set(n_data['title']))[:10])

3628
["City Slickers II: The Legend of Curly's Gold (1994)", 'Roadside Prophets (1992)', 'Indiana Jones and the Temple of Doom (1984)', 'Around the World in 80 Days (1956)', 'Glimmer Man, The (1996)', 'Bastard Out of Carolina (1996)', 'Jungle Book, The (1967)', 'Glass Bottom Boat, The (1966)', "Cats Don't Dance (1997)", 'Sixth Sense, The (1999)']


In [18]:
print(views[:10])
print(max(views))
print(len(n_data))

[1680, 443, 611, 1223, 1599, 2252, 666, 1290, 707, 1650]
3211
836478


In [19]:
n_data['title'][66:68]

# 67 인덱스가 사라졌다

66    Jackal, The (1997)
68      Gladiator (2000)
Name: title, dtype: object

In [20]:
n_data[-1:]

,user_id,title,genre
1000208,6040,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


In [21]:
len(n_data)

836478

In [22]:
1000208 - 836478

163730

확인해보니까 null 값도 아니었다

도대체 뭐지?

왜 인덱스 값은 백만인데, 실제 길이는 80만인거지?
<br/>
확인해보니, 원래 요소는 80만개밖에 없다
<br/>
인덱스를 재정의해줘야 겠다
<br/>


In [23]:
n_data.index=list(range(836478))

인덱스 재정의하는 방법

https://kongdols-room.tistory.com/123

In [24]:
n_data[-1:]

,user_id,title,genre
836477,6040,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


n_data['views'] = list(n_data['title'])
cc = 0

for i in n_data['title']:
    c_views = 0
    for j in range(3211):
        if n_data['views'][c_views] == i:
            n_data['views'][c_views] = views[c_views]
        else:
            c_views+=1
    cc +=1
    print(cc)

In [69]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = n_data['user_id'].nunique()
num_artist = n_data['title'].nunique()

csr_data = csr_matrix((n_data.views, (n_data.user_id,n_data.title)), shape= (num_user, num_artist))
csr_data

ValueError: invalid literal for int() with base 10: "One Flew Over the Cuckoo's Nest (1975)"

In [43]:
from random import randrange

aaaa = pd.DataFrame({'sd':list(range(8)), 'sqa':list(range(8,16)), 'eqwr':randrange(1,8)})

from scipy.sparse import csr_matrix

aaaa.sort_values(by='sd' ,ascending=False)

num_user = aaaa['sd'].nunique()
num_artist = aaaa['eqwr'].nunique()

csr_matrix((aaaa.sqa, (aaaa.sd, aaaa.eqwr)), shape=(num_user, num_artist))


ValueError: column index exceeds matrix dimensions

## 회고

csr을 만들기 위해서는 숫자를 넣어야 될 거 같아 새로운 파일로 만들었습니다

결과 row index exceeds matrix dimensions 라는 에러만 뜨네요

https://stackoverflow.com/questions/46396450/sparse-matrix-valueerror-row-index-exceeds-matrix-dimensions

답변을 읽어도 모르겠습니다

설마 csr 매트릭스 제작하는 과정에서 이렇게 막힐 줄 몰랐습니다
<br/>
여튼 저는 영화 이름의 출현 빈도수를 조회수라 정의하고 조회수를 csr_matrix의 첫번째 매게변수로 넣어주고자 했습니다
<br/>
허나 조회수를 기입하는 단계가 엄청나게 오래걸렸고, 결국 이렇게 되었네요
<br/>
